In [1]:
import pandas as pd 
import numpy as np 
import preprocessing
%run tfidf.ipynb

pd.set_option('max_colwidth', 5000)

In [2]:
#leggi db stack overflow 
#answers = pd.read_csv("DB/Answers.csv", sep=",",low_memory = False ,encoding='latin-1')
#tags = pd.read_csv("DB/Tags.csv", sep=";", low_memory = False)
questions = pd.read_csv("DB/Questions.csv", sep=",", low_memory = False, encoding='latin-1')

In [3]:
# droppa le colonne data e owner user id che non servono e prendi solo le prime 100 righe
#answers = answers.head(100)
#answers.drop("CreationDate", axis=1, inplace=True)
#answers.drop("OwnerUserId", axis=1, inplace=True)

In [4]:
#raggruppa i tag per id di domanda, in modo da avere una lista di tag per ogni domanda 
#tags = tags.groupby('Id').agg(lambda x: x.tolist())
#tags = tags.head(100)

In [5]:
# droppa le colonne relative alle date che non servono e prendi solo le prime 100 righe
questions.drop("CreationDate", axis=1, inplace=True)
questions.drop("ClosedDate", axis=1, inplace = True)
questions = questions.head(10000)

In [6]:
body_of_questions = questions["Body"]
body_of_questions = preprocessing.remove_tags(body_of_questions)
#body_of_answers

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    I've written a database generation script in  and want to execute it in my  application:\n\n\n\nI execute this in Adobe AIR using the following methods:\n\n\n\nNo errors are generated, however only  exists It seems that it only looks at the first query (up to the semicolon- if I remove it, the query fails) Is there a way to call multiple queries in one 

In [7]:
#Prove di clear text


#body_of_answers = pd.Series(body_of_answers, dtype="string") 
#body_of_answers = clear_text(body_of_answers)
docs = questions['Body'].apply(lambda x: preprocessing.clear_text(x))
docs
docs.to_csv('DB/questions_cleared')

#body_of_answers = to_lower_case(body_of_answers)

#body_of_answers = body_of_answers.str.lower()
#body_of_answers



In [8]:
get_word_count_vec(docs)


In [18]:
df =  pd.read_csv("DB/questions_cleared.csv", sep=",", low_memory = False, encoding='latin-1')


#for i, row in df.iterrows('Body'):
#    print(type(row))

TypeError: iterrows() takes 1 positional argument but 2 were given